In [54]:
# packages
import os
import numpy as np
import pandas as pd
import pickle
import ray 
import matplotlib.pyplot as plt
import matplotlib as mpl
from multiprocessing import Pool
import joblib
from joblib import Parallel, delayed
from src.model_2state import solve_prep, solve_pre_jump_2state
from src.simulation_2d import simulation_2d, simulate_logkapital
from src.plots import plot_2S_ey1y2, plot_1S_vs_2S_ems, plot_1S_vs_2S_SCC, plot_2S_ey1y2_multi_λ, plot_1S_vs_2S_ems_multi_λ, plot_1S_vs_2S_SCC_multi_λ 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.interpolate import interp2d
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams["figure.edgecolor"] = "w"
mpl.rcParams['figure.facecolor'] = "w"
mpl.rcParams["savefig.dpi"] = 300

# Appendix C: when $Y$ has two states:

Recall in remark 1 in [section 2](sec2_UncertainClimateDynamics.ipynb),
we capture the initial rise in the emission responses by the following two-dimensional specification:

$$
dY_t^1  = Y_t^2 dt
$$

$$
dY_t^2 =   - \lambda Y_t^2 dt + \lambda \theta \mathcal{E} dt
$$

which implies the response to a pulse is:

$$
\theta \left[ 1 - \exp( - \lambda t) \right] {\mathcal E}_0
$$
 
A high value of $\lambda$ implies more rapid convergence to the limiting response $\theta  {\mathcal E}_0$.  This  approximation is intended as a simple representation of the dynamics where the second state variable can be thought of as an exponentially weighted average of current and past emissions.\footnote{In independent work, \cite{DietzVenmans:2019} and \cite{BarnettBrockHansen:2020} have used such simplified approximations within an explicit economic optimization framework.  The former contribution includes the initial rapid upswing in the impulse response functions.  The latter contribution  abstracts from this.   \citeauthor{BarnettBrockHansen:2020} instead explore ways to confront uncertainty, broadly-conceived, while using the Matthews approximation.}

## Step I: post jump HJB:

The post jump HJB for $\phi_m(y_1, y_2)$, $m = 1, \dots, M$:

$$
\begin{aligned}
0 = \max_{\mathcal{E}} \min_{\omega_\ell } & - \delta \phi_m(y_1, y_2) + \eta log(\mathcal{E}) \\
& + \frac{\partial \phi_m}{\partial y_1} y_2 + \frac{\partial \phi_m}{\partial y_2} \lambda (- y_2 + \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} \left(\gamma_1 + \gamma_2 y_1 + \gamma_3 (y_1 - \bar y)\mathbb{I}\{y_1>\bar y\} \right) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)
\end{aligned}
$$

First order condition for $\omega_\ell$, $\ell = 1, \dots, L$:

$$
    \omega_\ell \propto \pi_\ell^a \exp\left( -\frac{1}{\xi_a} \frac{\partial \phi_m}{\partial y_2}\lambda \theta_\ell \mathcal{E} \right), for \ell = 1, \dots, L
$$

and the first order condition for emission is:
$$
\mathcal{E} = - \cfrac{\eta}{\frac{\partial \phi_m }{\partial y_2} \lambda \sum_{\ell=1}^{L} \omega_\ell \theta_\ell}
$$


## Step II: pre jump HJB:

Given post jump value functions $\phi_m(y_1, y_2)$, $m = 1, 2, M$, solve the following HJB for pre-jump value function $\Phi(y_1, y_2)$:

$$
\begin{aligned}
0 = \max_{\mathcal{E}}\min_{\omega_\ell }  & - \delta \Phi(y_1, y_2) +  \eta log(\mathcal{E}) \\
& + \frac{\partial \Phi}{\partial y_1} y_2 + \frac{\partial \Phi}{\partial y_2} \lambda (- y_2+ \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} (\gamma_1 + \gamma_2 y_1 ) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)\\
& + \mathcal{J}(y_1) \sum_{m=1}^M g_m \pi_d^m ( \phi_m(\bar{y}_1, y_2) - \Phi(y_1, y_2)) \\
& + \xi_p \mathcal{J}(y_1) \sum_{m=1}^M \pi_d^m \left(1 - g_m + g_m \log (g_m)\right)
\end{aligned}
$$

Or solve the following HJB with a terminal condition:

$$
\begin{aligned}
0 = \max_{\mathcal{E}}\min_{\omega_\ell }  & - \delta \Phi(y_1, y_2) +  \eta log(\mathcal{E}) \\
& + \frac{\partial \Phi}{\partial y_1} y_2 + \frac{\partial \Phi}{\partial y_2} \lambda (- y_2+ \sum_{\ell = 1}^L 
\omega_\ell  \theta_\ell \mathcal{E}) \\ 
& + \frac{(\eta - 1)}{\delta} (\gamma_1 + \gamma_2 y_1 ) y_2 \\
& + \xi_a \sum_{\ell = 1}^L \omega_\ell (\log \omega_\ell - \log \pi^a_\ell)
\end{aligned}
$$

$$
\Phi(\bar y_1, y_2) \approx  - \xi_p \log \left (\sum_{m=1}^M \pi_m^p \exp\left[-\frac{1}{\xi_p }\phi_m(\bar y_1, y_2) \right] \right) 
$$

In what follows, we show emission, $Y$ behavior before temperature anomay reaches the upper bound of jump threshold, $2^o C$, conditioning on no jump.

The $\lambda$ we choose here are:
- $\lambda = 0.116$, corresponds to half life of six years;
- $\lambda = 1, 2, 5$. Here, we try to illustrate emission behavior as $\lambda$ increase.

## Example 1: Half life of six years

In [2]:
λ = np.log(2) / 6
# parameters
δ = 0.01
η = 0.032
ξ_a = 0.01
θ_list = pd.read_csv("./data/model144.csv", header=None)[0].to_numpy()
θ_list = θ_list/1000
θ = np.mean(θ_list)
σy = 1.2*θ
# damage function
y_bar = 2.
γ_1 = 0.00017675
γ_2 = 2*0.0022
γ_3_list = np.linspace(0., 1./3, 20)
ξ_r = 1.
# %%
# y_grid
y1_step = .04
y1_grid = np.arange(0., 4. + y1_step, y1_step)

y2_step = .001
y2_grid = np.arange(0., .05 + y2_step, y2_step)

In [3]:
args_list  = []
for γ_3_i in γ_3_list:
    args_iter = (y1_grid, y2_grid, γ_3_i, θ_list, (δ, η, γ_1, γ_2, y_bar, λ, ξ_a), 1e-6, 1., 1000, 0.05)
    args_list.append(args_iter)

if not os.path.exists(f"./data/res_list_{λ}.pickle"):
    with Pool() as p:
        res_list = p.starmap(solve_prep, args_list)

    with open(f"./data/res_list_{λ}.pickle", "wb") as handle:
        pickle.dump(res_list, handle)

# step II HJB:
with open(f"./data/res_list_{λ}.pickle", "rb") as file:
    res_list = pickle.load(file)

if not os.path.exists(f"./data/res_{λ}_{ξ_r}.pickle"):
    args_pre_jump = (δ, η, θ_list,  γ_1, γ_2, γ_3_list, ξ_a, ξ_r)
    res = solve_pre_jump_2state(res_list, args_pre_jump, ε=0.1)
    with open(f"./data/res_{λ}_{ξ_r}.pickle", "wb") as handle:
        pickle.dump(res, handle)

In [4]:
#load results
res = pickle.load(open(f"./data/res_{λ}_{ξ_r}.pickle", "rb"))

In [13]:
def simulation_2d(res, θ=1.86/1000., y1_0=1.1, y2_0=1.86/1000, T=100):
    y1_grid = res["y1"]
    y2_grid = res["y2"]
    e_grid = res["ems"]
    λ = res["λ"]
    e_fun = interp2d(y1_grid, y2_grid, e_grid.T)
    Et = np.zeros(T+1)
    y1t = np.zeros(T+1)
    y2t = np.zeros(T+1)
    for i in range(T+1):
        Et[i] = e_fun(y1_0, y2_0)
        y1t[i] = y1_0
        y2t[i] = y2_0
        y2_0 = y2_0 - λ*y2_0 + λ*θ*Et[i] 
        y1_0 = y1_0 + y2_0
    return Et, y1t, y2t

In [6]:
et_prejump, y1t_prejump, y2t_prejump = simulation_2d(res, θ=np.mean(θ_list), y1_0 = 1.1, y2_0=np.mean(θ_list), T=110)

In [7]:
simul = {
    "et": et_prejump,
    "y1t": y1t_prejump,
    "y2t": y2t_prejump,
}

pickle.dump(simul, open(f"data/simul_{λ}.pickle", "wb"))

In [36]:
fig = plot_2S_ey1y2(simul)
fig.update_xaxes(zeroline=True)
fig.update_yaxes(zeroline=True, rangemode="tozero")

Compare the emission trajectories with the one state model:

In [37]:
et_1state = np.load("data/et_1state_cal.npy")
plot_1S_vs_2S_ems(et_1state, simul)

In the following plot, we show the social cost of carbon conditioning on no jump happening with comparison to the model with one state.

In [38]:
# capital related parameters
invkap = 0.09
α = 0.115
αₖ = - 0.043
σₖ = 0.0095
κ = 6.667
k0 = 85/α

In [43]:
Kt = simulate_logkapital(invkap, αₖ, σₖ, κ,  k0, T=111)
MC = δ*(1-η)/((α - invkap)*np.exp(Kt))
scc = η*(α - invkap)*np.exp(Kt)/(1-η)/et_prejump*1000
scc_1 = η*(α - invkap)*np.exp(Kt[:len(et_1state)])/(1-η)/et_1state*1000

In [45]:
# TODO: scc comparison
fig = plot_1S_vs_2S_SCC(et_1state, scc[:101], scc_1)
fig.update_xaxes(showline=True)
fig.update_yaxes(rangemode="tozero")

# Example 2: increasing $\lambda$

For the purpose of illustration, we choose $\lambda = 1, 2, 5$ to show what happens if the half life decreases.

In [46]:
res_dict = {}
λ_list = [1., 2., 5.]

for λ in λ_list:
    if not os.path.exists(f"./data/res_list_{λ}.pickle"):
        res_list = [delayed(solve_prep)(y1_grid, y2_grid, γ_3_i, θ_list, 
                                      (δ, η, γ_1, γ_2, y_bar, λ, ξ_a), 1e-6, 0.05, 10000, 0.05) for γ_3_i in γ_3_list]

        with open(f"./data/res_list_{λ}.pickle", "wb") as handle:
            pickle.dump(res_list, handle)

    # step II HJB:
    with open(f"./data/res_list_{λ}.pickle", "rb") as file:
        res_list = pickle.load(file)
    args_pre_jump = (δ, η, θ_list,  γ_1, γ_2, γ_3_list, ξ_a, ξ_r)
    res = solve_pre_jump_2state(res_list, args_pre_jump, ε=.1)

    with open(f"./data/res_{λ}_{ξ_r}.pickle", "wb") as handle:
        pickle.dump(res, handle)
    res_dict[λ] = res

KeyboardInterrupt: 

In [ ]:
number_of_cpu = joblib.cpu_count()
λ = 2.
delayed_funcs = [delayed(solve_prep)(y1_grid, y2_grid, γ_3_i, θ_list, 
                                      (δ, η, γ_1, γ_2, y_bar, λ, ξ_a), 1e-6, 0.01, 10000, 0.02) for γ_3_i in γ_3_list]
parallel_pool = Parallel(n_jobs=number_of_cpu)
res_list = parallel_pool(delayed_funcs)

Time elapsed: 0.005118
Time elapsed: 0.005451
Time elapsed: 0.004959
Time elapsed: 0.00391
Time elapsed: 0.00479
Time elapsed: 0.004083
Time elapsed: 0.004496
Time elapsed: 0.003464
Time elapsed: 0.004347
Time elapsed: 0.003661
Time elapsed: 0.003142
Time elapsed: 0.002924
Time elapsed: 0.003077
Time elapsed: 0.002986
Time elapsed: 0.003316
Time elapsed: 0.002867
Time elapsed: 0.002248
Time elapsed: 0.002712
Time elapsed: 0.002732
Time elapsed: 0.002477
Time elapsed: 0.002714
Time elapsed: 0.00255
Time elapsed: 0.002537
Time elapsed: 0.002973
Time elapsed: 0.002396
Time elapsed: 0.002338
Time elapsed: 0.002611
Time elapsed: 0.002603
Time elapsed: 0.002539
Time elapsed: 0.00266
Time elapsed: 0.002328
Time elapsed: 0.002551
Time elapsed: 0.003838
Time elapsed: 0.003909
Time elapsed: 0.005986
Time elapsed: 0.005874
Time elapsed: 0.00634
Time elapsed: 0.00438
Time elapsed: 0.004145
Time elapsed: 0.00527
Time elapsed: 0.00501
Time elapsed: 0.004772
Time elapsed: 0.00364
Time elapsed: 0.0027

Time elapsed: 0.008731
Time elapsed: 0.00539
Time elapsed: 0.005688
Time elapsed: 0.005791
Time elapsed: 0.005698
Time elapsed: 0.00598
Time elapsed: 0.009421
Time elapsed: 0.009731
Time elapsed: 0.010455
Time elapsed: 0.006101
Time elapsed: 0.009784
Time elapsed: 0.004445
Time elapsed: 0.007191
Time elapsed: 0.008243
Time elapsed: 0.009216
Time elapsed: 0.011615
Time elapsed: 0.00391
Time elapsed: 0.00419
Time elapsed: 0.003915
Time elapsed: 0.006058
Time elapsed: 0.003517
Time elapsed: 0.003568
Time elapsed: 0.003532
Time elapsed: 0.00408
Time elapsed: 0.003578
Time elapsed: 0.003806
Time elapsed: 0.003903
Time elapsed: 0.002835
Time elapsed: 0.002742
Time elapsed: 0.002836
Time elapsed: 0.006697
Time elapsed: 0.007209
Time elapsed: 0.006774
Time elapsed: 0.007399
Time elapsed: 0.007583
Time elapsed: 0.002694
Time elapsed: 0.007371
Time elapsed: 0.009525
Time elapsed: 0.009495
Time elapsed: 0.003208
Time elapsed: 0.010814
Time elapsed: 0.009007
Time elapsed: 0.003233
Time elapsed: 0.

Time elapsed: 0.021076
Time elapsed: 0.003091
Time elapsed: 0.002358
Time elapsed: 0.002398
Time elapsed: 0.002964
Time elapsed: 0.002669
Time elapsed: 0.002931
Time elapsed: 0.002361
Time elapsed: 0.002995
Time elapsed: 0.003105
Time elapsed: 0.00361
Time elapsed: 0.018126
Time elapsed: 0.017978
Time elapsed: 0.018146
Time elapsed: 0.019083
Time elapsed: 0.002613
Time elapsed: 0.002705
Time elapsed: 0.002728
Time elapsed: 0.002473
Time elapsed: 0.002331
Time elapsed: 0.002727
Time elapsed: 0.002367
Time elapsed: 0.002383
Time elapsed: 0.018916
Time elapsed: 0.018765
Time elapsed: 0.019231
Time elapsed: 0.019834
Time elapsed: 0.019961
Time elapsed: 0.020794
Time elapsed: 0.002592
Time elapsed: 0.003593
Time elapsed: 0.003442
Time elapsed: 0.004408
Time elapsed: 0.004013
Time elapsed: 0.003926
Time elapsed: 0.002725
Time elapsed: 0.002702
Time elapsed: 0.002811
Time elapsed: 0.002669
Time elapsed: 0.020532
Time elapsed: 0.018897
Time elapsed: 0.019627
Time elapsed: 0.018839
Time elapsed

Time elapsed: 0.002221
Time elapsed: 0.025533
Time elapsed: 0.024863
Time elapsed: 0.024308
Time elapsed: 0.02349
Time elapsed: 0.015509
Time elapsed: 0.014588
Time elapsed: 0.014423
Time elapsed: 0.003032
Time elapsed: 0.00281
Time elapsed: 0.002779
Time elapsed: 0.002457
Time elapsed: 0.022151
Time elapsed: 0.023462
Time elapsed: 0.002323
Time elapsed: 0.002391
Time elapsed: 0.002374
Time elapsed: 0.00218
Time elapsed: 0.002247
Time elapsed: 0.015798
Time elapsed: 0.023892
Time elapsed: 0.025874
Time elapsed: 0.026926
Time elapsed: 0.023699
Time elapsed: 0.002045
Time elapsed: 0.027676
Time elapsed: 0.026359
Time elapsed: 0.025118
Time elapsed: 0.002607
Time elapsed: 0.001736
Time elapsed: 0.003021
Time elapsed: 0.015975
Time elapsed: 0.016922
Time elapsed: 0.018571
Time elapsed: 0.003326
Time elapsed: 0.025754
Time elapsed: 0.002987
Time elapsed: 0.002762
Time elapsed: 0.00273
Time elapsed: 0.026122
Time elapsed: 0.003192
Time elapsed: 0.002594
Time elapsed: 0.002439
Time elapsed: 0

Time elapsed: 0.002995
Time elapsed: 0.032843
Time elapsed: 0.003182
Time elapsed: 0.002822
Time elapsed: 0.002835
Time elapsed: 0.002677
Time elapsed: 0.030808
Time elapsed: 0.037738
Time elapsed: 0.002823
Time elapsed: 0.002618
Time elapsed: 0.00264
Time elapsed: 0.002416
Time elapsed: 0.025293
Time elapsed: 0.032517
Time elapsed: 0.028922
Time elapsed: 0.035749
Time elapsed: 0.027506
Time elapsed: 0.025304
Time elapsed: 0.027336
Time elapsed: 0.027119
Time elapsed: 0.039629
Time elapsed: 0.003173
Time elapsed: 0.002912
Time elapsed: 0.004152
Time elapsed: 0.002909
Time elapsed: 0.027086
Time elapsed: 0.033077
Time elapsed: 0.003451
Time elapsed: 0.003082
Time elapsed: 0.003186
Time elapsed: 0.003199
Time elapsed: 0.002413
Time elapsed: 0.031876
Time elapsed: 0.036327
Time elapsed: 0.002992
Time elapsed: 0.002694
Time elapsed: 0.002655
Time elapsed: 0.002629
Time elapsed: 0.030546
Time elapsed: 0.032804
Time elapsed: 0.030962
Time elapsed: 0.030235
Time elapsed: 0.029183
Time elapsed

...

Time elapsed: 0.002348
Time elapsed: 0.001247
Time elapsed: 0.001714
Time elapsed: 0.002129
Time elapsed: 0.001733
Time elapsed: 0.001619
Time elapsed: 0.001734
Time elapsed: 0.001717
Time elapsed: 0.001455
Time elapsed: 0.001902
Time elapsed: 0.001557
Time elapsed: 0.002239
Time elapsed: 0.001882
Time elapsed: 0.005618
Time elapsed: 0.003895
Time elapsed: 0.004267
Time elapsed: 0.004091
Time elapsed: 0.004029
Time elapsed: 0.003193
Time elapsed: 0.003087
Time elapsed: 0.003086
Time elapsed: 0.00262
Time elapsed: 0.002513
Time elapsed: 0.001481
Time elapsed: 0.002005
Time elapsed: 0.001398
Time elapsed: 0.001837
Time elapsed: 0.001639
Time elapsed: 0.00191
Time elapsed: 0.001802
Time elapsed: 0.00187
Time elapsed: 0.001565
Time elapsed: 0.001475
Time elapsed: 0.001939
Time elapsed: 0.002152
Time elapsed: 0.002776
Time elapsed: 0.003196
Time elapsed: 0.002704
Time elapsed: 0.003428
Time elapsed: 0.003821
Time elapsed: 0.003929
Time elapsed: 0.003764
Time elapsed: 0.00328
Time elapsed: 0

Time elapsed: 0.002158
Time elapsed: 0.001536
Time elapsed: 0.001396
Time elapsed: 0.00193
Time elapsed: 0.001717
Time elapsed: 0.002023
Time elapsed: 0.001832
Time elapsed: 0.001567
Time elapsed: 0.002167
Time elapsed: 0.002341
Time elapsed: 0.002055
Time elapsed: 0.003143
Time elapsed: 0.00324
Time elapsed: 0.003523
Time elapsed: 0.003466
Time elapsed: 0.003758
Time elapsed: 0.00378
Time elapsed: 0.003247
Time elapsed: 0.002826
Time elapsed: 0.002834
Time elapsed: 0.002804
Time elapsed: 0.002805
Time elapsed: 0.002628
Time elapsed: 0.001631
Time elapsed: 0.001983
Time elapsed: 0.002094
Time elapsed: 0.002055
Time elapsed: 0.001906
Time elapsed: 0.002483
Time elapsed: 0.001703
Time elapsed: 0.001799
Time elapsed: 0.00247
Time elapsed: 0.002577
Time elapsed: 0.002035
Time elapsed: 0.003956
Time elapsed: 0.00394
Time elapsed: 0.004353
Time elapsed: 0.003508
Time elapsed: 0.003911
Time elapsed: 0.003197
Time elapsed: 0.002351
Time elapsed: 0.00268
Time elapsed: 0.002926
Time elapsed: 0.0

Time elapsed: 0.001505
Time elapsed: 0.001936
Time elapsed: 0.001972
Time elapsed: 0.001938
Time elapsed: 0.001444
Time elapsed: 0.001614
Time elapsed: 0.001726
Time elapsed: 0.001617
Time elapsed: 0.00296
Time elapsed: 0.002562
Time elapsed: 0.002196
Time elapsed: 0.004696
Time elapsed: 0.003563
Time elapsed: 0.003789
Time elapsed: 0.003887
Time elapsed: 0.004006
Time elapsed: 0.002425
Time elapsed: 0.002803
Time elapsed: 0.002425
Time elapsed: 0.002918
Time elapsed: 0.00259
Time elapsed: 0.002518
Time elapsed: 0.001701
Time elapsed: 0.001569
Time elapsed: 0.001382
Time elapsed: 0.002043
Time elapsed: 0.002155
Time elapsed: 0.002147
Time elapsed: 0.002048
Time elapsed: 0.002117
Time elapsed: 0.002349
Time elapsed: 0.001668
Time elapsed: 0.002921
Time elapsed: 0.003648
Time elapsed: 0.003911
Time elapsed: 0.004191
Time elapsed: 0.003728
Time elapsed: 0.003977
Time elapsed: 0.003082
Time elapsed: 0.00233
Time elapsed: 0.002716
Time elapsed: 0.002178
Time elapsed: 0.002957
Time elapsed: 

Time elapsed: 0.003194
Time elapsed: 0.002775
Time elapsed: 0.002574
Time elapsed: 0.002503
Time elapsed: 0.002582
Time elapsed: 0.001392
Time elapsed: 0.001435
Time elapsed: 0.001429
Time elapsed: 0.001372
Time elapsed: 0.001347
Time elapsed: 0.002091
Time elapsed: 0.002211
Time elapsed: 0.001608
Time elapsed: 0.001798
Time elapsed: 0.002839
Time elapsed: 0.002611
Time elapsed: 0.004126
Time elapsed: 0.005312
Time elapsed: 0.004035
Time elapsed: 0.004406
Time elapsed: 0.004605
Time elapsed: 0.003221
Time elapsed: 0.00338
Time elapsed: 0.003019
Time elapsed: 0.00262
Time elapsed: 0.002905
Time elapsed: 0.002855
Time elapsed: 0.001836
Time elapsed: 0.002074
Time elapsed: 0.001347
Time elapsed: 0.001468
Time elapsed: 0.001748
Time elapsed: 0.001995
Time elapsed: 0.001523
Time elapsed: 0.00195
Time elapsed: 0.001766
Time elapsed: 0.001574
Time elapsed: 0.001923
Time elapsed: 0.00436
Time elapsed: 0.00532
Time elapsed: 0.00396
Time elapsed: 0.003534
Time elapsed: 0.003115
Time elapsed: 0.0

Time elapsed: 0.001511
Time elapsed: 0.004547
Time elapsed: 0.002997
Time elapsed: 0.002218
Time elapsed: 0.002316
Time elapsed: 0.002848
Time elapsed: 0.002177
Time elapsed: 0.002905
Time elapsed: 0.005419
Time elapsed: 0.002865
Time elapsed: 0.002932
Time elapsed: 0.002897
Time elapsed: 0.001345
Time elapsed: 0.001384
Time elapsed: 0.002157
Time elapsed: 0.001806
Time elapsed: 0.001333
Time elapsed: 0.001298
Time elapsed: 0.001742
Time elapsed: 0.00165
Time elapsed: 0.001736
Time elapsed: 0.002422
Time elapsed: 0.002303
Time elapsed: 0.002681
Time elapsed: 0.001704
Time elapsed: 0.003054
Time elapsed: 0.003197
Time elapsed: 0.002993
Time elapsed: 0.004266
Time elapsed: 0.003838
Time elapsed: 0.002731
Time elapsed: 0.002394
Time elapsed: 0.003246
Time elapsed: 0.002786
Time elapsed: 0.001421
Time elapsed: 0.001562
Time elapsed: 0.002306
Time elapsed: 0.001287
Time elapsed: 0.001897
Time elapsed: 0.001668
Time elapsed: 0.001812
Time elapsed: 0.002003
Time elapsed: 0.001979
Time elapsed

In [ ]:
with open(f"./data/res_list_{λ}.pickle", "wb") as handle:
    pickle.dump(res_list, handle)

In [ ]:
args_pre_jump = (δ, η, θ_list,  γ_1, γ_2, γ_3_list, ξ_a, ξ_r)
res = solve_pre_jump_2state(res_list, args_pre_jump, ε=.1)

with open(f"./data/res_{λ}_{ξ_r}.pickle", "wb") as handle:
    pickle.dump(res, handle)

In [47]:
simul_list = {}
for λ in λ_list:
    res_temp=pickle.load(open(f"./data/res_{λ}_{ξ_r}.pickle", "rb"))
    et_prejump, y1t_prejump, y2t_prejump = simulation_2d(res_temp,
                                                  θ=np.mean(θ_list),
                                                  y1_0 = 1.1,
                                                  y2_0=np.mean(θ_list),
                                                  T=110
                                                 )
    simul = {
    "et": et_prejump,
    "y1t": y1t_prejump,
    "y2t": y2t_prejump,
    }
    simul_list[λ] = simul

pickle.dump(simul, open(f"data/simul_{λ}", "wb"))

In [48]:
# plot, emission, y1, y2, button, λ = 1, 2, 5
plot_2S_ey1y2_multi_λ(simul_list, λ_list)

In [49]:
#  emission, λ = 1,2,5 and 1 state result
plot_1S_vs_2S_ems_multi_λ(et_1state, simul_list, λ_list)

In [50]:
scc_list=np.zeros((3, 111))
for i in range(len(λ_list)):
    scc_list[i] = η*(α - invkap)*np.exp(Kt)/(1-η)/simul_list[λ_list[i]]["et"]*1000

In [51]:
# scc λ = 1, 2, 5 and 1 state
plot_1S_vs_2S_SCC_multi_λ(et_1state, scc_list, scc_1, λ_list)